# Markovian simulation

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Markov = True

In [ ]:
dx = [.5,.5]
nx = [400,400]
db = gl.DbGrid.create(nx, dx)

In [ ]:
scale = 2
sill = 2.
param=2
mesh = gl.MeshETurbo(nx,dx)

ranges = [1 * scale, 1.8*scale]
angles = [30.,0]
if Markov :
    model = gl.Model.createFromParam(gl.ECov.MARKOV,ranges=ranges,angles = angles,
                                     sill=sill,flagRange=False)
    cova = model.getCova(0)
    cova.setMarkovCoeffsBySquaredPolynoms([1,-1.5],[1,-1],0.001)
    #coeffs = np.array(cova.getMarkovCoeffs())
    #for i in range(len(coeffs)):
    #    coeffs[i]*= scale**(2*i)
    #cova.setMarkovCoeffs(coeffs)
else :
    model = gl.Model.createFromParam(gl.ECov.BESSEL_K,param=param,
                                     ranges=ranges,sill=sill,
                                     angles =angles, flagRange=False)


In [ ]:
model

In [ ]:
Q = gl.PrecisionOp(mesh,model)

In [ ]:
cova = model.getCova(0)
cova.getMarkovCoeffs()

In [ ]:
cova.getCorrec()

In [ ]:
cova.getRange()

In [ ]:
indx = int(nx[0]/2)
indy = int(nx[1]/2)
ind = indy + nx[1] * indx
covy = np.array(Q.evalCov(ind)).reshape(nx)

In [ ]:
xx = np.arange(len(covy)/2)*dx[0]
if not Markov :
    covM = [model.eval(0,0,gl.SpacePoint([0,0]),gl.SpacePoint([i,0])) for i in xx]
    plt.plot(xx,covM,"--",label = "Covariance")
plt.plot(xx,covy[indx,indy:(indy+nx[1])],label = "discretized")#/covy[indx,indx])
covy[indx,indx]
ax = plt.legend()
plt.show()
plt.imshow(covy.reshape(nx),origin = "lower")

In [ ]:
dbg = gl.DbGrid.create(nx,dx)
x = Q.simulate()[0]
dbg["simu"]=x
gp.plot(dbg)

In [ ]:
N = 2**8
cova = model.getCova(0)
mH1 = 3.5 * np.max(cova.getRanges())
mH2 = 3.5 * np.max(cova.getRanges())

result = np.array(cova.evalCovFFT([mH1,mH2],N).getValues())
res = np.array(result).reshape((N,N))

X1 = np.linspace(-mH1,mH1,N) 
X2 = np.linspace(-mH2,mH2,N) 
plt.imshow(res,extent = [-mH1,mH1,-mH2,mH2],origin = "lower")
#plt.colorbar()


In [ ]:
plt.plot(X1 , res[int(N/2),:],label="FFT")

uu = covy[indx,indy:(indy+100)]
plt.plot(np.arange(len(uu))*dx[0],uu,"--",label = "Discretized covariance")
ax = plt.legend()

In [ ]:
res[int(N/2),int(N/2)]/uu[0]